<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Przetwarzanie danych wektorowych</font>

<font size="4">Krzysztof Dyba</font>

In [4]:
import os
from qgis.core import QgsVectorLayer

# wczytanie warstwy wektorowej
sciezka_do_pliku = os.path.join("algorytmy-geoprzestrzenne", "dane", "powiaty.gpkg")
wektor = QgsVectorLayer(sciezka_do_pliku, "powiaty", "ogr")
print(wektor.isValid())

True


In [44]:
print(wektor.featureCount())
print(wektor.fields().count())
print(wektor.fields().names())

35
6
['fid', 'JPT_KOD_JE', 'JPT_NAZWA_', 'IIP_IDENTY', 'IIP_WERSJA', 'pole_km2']


# Dostęp do obiektów

In [38]:
from itertools import islice

# iteracja po obiektach
for feature in islice(wektor.getFeatures(), 5):
    # dostęp do atrybutów obiektów
    attrs = feature.attributes()
    # dostęp do geometrii obiektów
    geom = feature.geometry()
    
    print(attrs)
    # print(geom)

[1, '3012', 'powiat krotoszyński', 'dc54ce6a-5272-45fc-a49c-824dbf170f2c', '2024-09-05T13:41:42+02:00', 712.7437283077061]
[2, '3002', 'powiat czarnkowsko-trzcianecki', 'b2e00ebb-412b-4ae8-9afc-fe68f65bb1ac', '2021-05-11T10:49:15+02:00', 1806.6288683868954]
[3, '3009', 'powiat kolski', '06a4a975-dc89-4ac3-9d51-cf188532908a', '2012-09-27T08:59:03+02:00', 1009.273272845488]
[4, '3029', 'powiat wolsztyński', '6f988fb3-2b2c-467e-9d07-c512d9215c05', '2024-08-27T13:56:13+02:00', 679.340479768619]
[5, '3026', 'powiat śremski', '2fc58bb6-d095-4f37-8f10-874797cb97fb', '2024-09-03T13:42:56+02:00', 573.5639814826102]


# Obliczanie powierzchni

W celu obliczenia powierzchni poligonów musimy wykonać pętle po wszystkich obiektach znajdujących się w warstwie i uzyskać dostęp do ich geometrii. Następnie, należy wykorzystać metodę `area()`, która oblicza pole powierzchni w jednostkach układu współrzędnych warstwy (zazwyczaj są to metry kwadratowe) w układzie planarnym (kartezjańskim). Opcjonalnie, możemy dokonać konwersji jednostek, np. na kilometry kwadratowe czy hektary. W niniejszym przykładzie wszystkie obiekty w warstwie posiadają geometrie.

In [33]:
for feature in islice(wektor.getFeatures(), 5):
    area = feature.geometry().area()
    area = area / 1000**2 # konwersja na km^2
    print(area)

712.7437283077061
1806.6288683868954
1009.273272845488
679.340479768619
573.5639814826102


# Dodawanie atrybutów

In [7]:
from qgis.core import QgsField
from qgis.PyQt.QtCore import QMetaType

# dodanie nowego atrybutu do tabeli
new_field = QgsField("pole_km2", QMetaType.Type.Double) # typ zmiennoprzecinkowy
wektor.dataProvider().addAttributes([new_field])
wektor.updateFields()

# rozpoczęcie trybu edycji warstwy
wektor.startEditing()

# indeks nowego atrybutu
area_idx = wektor.fields().indexOf("pole_km2")

# obliczenie powierzchni dla każdego obiektu
for feature in wektor.getFeatures():
    area = feature.geometry().area()
    area = area / 1000**2
    # wprowadzenie wartości do atrybutu obiektu
    wektor.changeAttributeValue(feature.id(), area_idx, area)

# zapisanie zmian
wektor.commitChanges()

True

In [34]:
# weryfikacja
print("pole_km2" in wektor.fields().names())

for feature in islice(wektor.getFeatures(), 5):
    print(feature.attribute("pole_km2"))

True
712.7437283077061
1806.6288683868954
1009.273272845488
679.340479768619
573.5639814826102


# Wybieranie obiektów

## Atrybut

W zasadzie są dwa sposoby: `selectByExpression`, `QgsFeatureRequest`.

In [40]:
# zapytanie
expression = '"pole_km2" > 1300'

# Use QgsFeatureRequest with the expression
wektor.selectByExpression(expression)

selected_features = wektor.selectedFeatures()
print(len(selected_features)) # liczba obiektów

for feature in selected_features:
    pass
    # print(feature.id()) # ID obiektów

4


In [ ]:
# wybierz obiekty na mapie (działa tylko w QGIS)
# from qgis.core import QgsProject
# QgsProject.instance().addMapLayer(wektor)
# wektor.selectByIds([feature.id() for feature in selected_features])

In [31]:
from qgis.core import QgsExpression, QgsFeatureRequest

expression = QgsExpression('"pole_km2" > 1300')

request = QgsFeatureRequest(expression)
selected_features = wektor.getFeatures(request)
for feature in selected_features:
    pass
    # print(feature.attributes())

In [30]:
from qgis.core import QgsFeatureRequest

expression = '"pole_km2" > 1300'

# stworzenie żądania używając zapytania
request = QgsFeatureRequest().setFilterExpression(expression)

for feature in wektor.getFeatures(request):
    pass
    # print(feature.attributes())

## Lokalizacja

In [13]:
from qgis.core import QgsRectangle

# wybierz obiekty używając prostokątu
rect = QgsRectangle(340000, 490000, 380000, 520000) # xmin, ymin, xmax, ymax
wektor.selectByRect(rect)

selected_features = wektor.selectedFeatures()
for feature in selected_features:
    pass
    # print(feature.attributes())

# Tworzenie nowej warstwy

In [23]:
from qgis.core import QgsGeometry

# typ geometrii i układ współrzędnych
geometry_type = "Polygon"
crs = "EPSG:2180"

# stwórz nową warstwę wektorową
layer = QgsVectorLayer(geometry_type+"?crs="+crs, "Nowa warstwa", "memory")
print(layer.isValid())

# rozpoczęcie trybu edycji warstwy
layer.startEditing()

# stwórz atrybuty
fields = [
    QgsField("ID", QMetaType.Type.Int),          # ID obiektu
    QgsField("nazwa", QMetaType.Type.QString),   # nazwa obiektu
    QgsField("wartosc", QMetaType.Type.Double),  # wartość
]

# dodaj atrybuty do warstwy
provider = layer.dataProvider()
provider.addAttributes(fields)
layer.updateFields()

# stwórz obiekt i uzupełnij atrybut
feature = QgsFeature()
polygon_wkt = QgsGeometry.fromRect(rect).asWkt()
polygon_geometry = QgsGeometry.fromWkt(polygon_wkt)
feature.setGeometry(polygon_geometry)
feature.setAttributes([1, "Prostokąt", 999])

# dodaj obiekt do warstwy
layer.addFeature(feature)
# zaktualizuj zasięg warstwy
layer.updateExtents()

# zapisanie zmian
layer.commitChanges()

# wyświetlenie w QGIS
# QgsProject.instance().addMapLayer(layer)

True


# Łączenie obiektów (dissolve)

De facto można zrobić to na dwa sposoby:
1. pętla
2. `processing.run("native:dissolve", parameters)`

# Tworzenie buforów

To też można zrobić na dwa sposoby.

In [9]:
from qgis.core import QgsFeature

# stworzenie pustej warstwy w pamięci
buffer_layer = QgsVectorLayer("Polygon?crs=" + wektor.crs().authid(), "Bufor", "memory")
buffer_provider = buffer_layer.dataProvider()

# odległość buforu w jednostkach warstwy (m)
buffer_distance = 1000.0

features = []
for feature in wektor.getFeatures():
    geom = feature.geometry()
    # stworzenie geometrii bufora
    buffer_geom = geom.buffer(buffer_distance, segments = 30)
    # stworzenie nowego obiektu
    buffer_feature = QgsFeature()
    # ustawienie geometrii
    buffer_feature.setGeometry(buffer_geom)
    # dodanie obiektu do listy
    features.append(buffer_feature)

# dodanie buforów do warstwy wektorowej
buffer_provider.addFeatures(features)
buffer_layer.updateExtents()

# wyświetlenie w QGIS
# QgsProject.instance().addMapLayer(buffer_layer)

ZADANIA:
- oblicz długość granic i dodaj jako nowy atrybut do warstwy
- napisz funkcję, która obliczy podstawowe statystki opisowe i zastosuj ją dla powierzchni oraz długości
- stwórz nową warstwę z obliczonymi centroidami dla powiatów

TODO:
- Buffering Features (Buffer Creation)
- Reprojecting Layer
- Use QGIS vector tools (Processing Toolbox)